# Import Agg Performance Nationwide

In [1]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import configparser

## Parameter

In [2]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

username = config['TDMDBPR']['username']
password = config['TDMDBPR']['password']
db = config['TDMDBPR']['db']
host = config['TDMDBPR']['host']
port = config['TDMDBPR']['port']
as_week = config['PARAM']['as_week']

In [3]:
raw_dir = f"../data/raw"
raw_file_name = f'Raw-Agg Perf Nationwide_Wk {as_week}'

## Query

In [4]:
conStr = f'{username}/{password}@{host}:{port}/{db}'
conn = cx_Oracle.connect(conStr)
print(f'{db} : Connected')
cur = conn.cursor()

v_week = (as_week, )


try:
    # print("Database version:", conn.version)
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, COMP_CD, METRIC_CD, METRIC_NAME, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACH_SNAP, GAP_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, ACH_AGG, GAP_AGG, WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT, RR, RR_ACH, WTD, MTD, QTD, YTD, CENTER, METRIC_GRP, PRODUCT_GRP, SEQ, PRIORITY, REMARK, AREA_TYPE, AREA_CD, AREA_NAME, REMARK_ACTUAL, REMARK_TARGET, PPN_TM, METRIC_NAME_GROUP
        FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK
        WHERE AREA_TYPE = 'P'
    """
    
    cur.execute(sqlTxt)

    rows = cur.fetchall()

    print(f'\nDone !!!')
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{db} : Disconnected')

TDMDBPR : Connected

Execute query as Week 2023033...

Done !!!

TDMDBPR : Disconnected


In [5]:
df.shape
# df.columns
# df.dtypes
# df.describe()

# df.head()

(60123, 54)

## Generate Output file

In [6]:
op_raw_file = f'{raw_dir}/{raw_file_name}.csv'

df.to_csv(f'{op_raw_file}', index=False, encoding='utf-8')

print(f'\n"{op_raw_file}" is generated')


"../data/raw/Raw-Agg Perf Nationwide_Wk 2023033.csv" is generated


In [7]:
cmd_rm = f'rm {raw_dir}/Raw-Agg.zip'
cmd_zip = f'zip {raw_dir}/Raw-Agg.zip {raw_dir}/Raw-Agg*.csv'

os.system(cmd_rm)
os.system(cmd_zip)

0